In [224]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import time
from W_settings import *
import datetime
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import os


In [225]:
separation_date = '31/12/2010'
begin_test = '01/01/2015'
test_date = '29/11/2014'
begin_date = '01/01/2005'
final_date = '31/12/2015'

In [226]:
def get_season(dt):
    # it returnts string
    #print('type of dt:' + str(type(dt)))
    #print(dt)
    dt = dt.date()
    if (dt.month < 3):
        return 'H'
    elif (dt.month < 6):
        return 'P'
    elif (dt.month < 9):
        return "E"
    elif (dt.month < 12):
        return "A"
    else:
        return "H"


In [227]:
def prepare_data(df_temp, keep_season=False, keep_wind=False):
    df_temp = df_temp.drop(["Date"], axis=1).dropna(axis=0, how="any")

    """
    for col in salouel_data.columns:
        train_data[col] = train_data[col].apply(scale_transformer, args=(settings_array[col],))

    for col in test_data.columns:
        test_data[col] = test_data[col].apply(scale_transformer, args=(settings_array[col],))
    """
    
    #df_temp = pd.get_dummies(train_data)
    #print(test_data.info())
    print("Dataframes saved...")
    return df_temp



In [228]:
cols = ['Date', 'PM10', 'RR', 'TN', 'TX', 'TM', 'PMERM', 'FFM', 'UM', 'GLOT', 'DXY']


In [229]:

def preprocessing_data(df):
    df = df.iloc[1:]
    df.columns = cols
    df = df.drop(["Date", 'NO2', 'O3'], axis=1)
    df = df.astype(float)
    #df = df.dropna(axis=0, how="any")
    return df


In [232]:
from pandas import read_csv
salouel_raw_data = pd.read_csv("./propositio/salou.csv", header=None, sep=';', decimal=',')

#salouel_raw_data = pd.read_csv("df_roth.csv", header=None, sep=';', decimal=',')


creil_raw_data = pd.read_csv("./propositio/creil.csv", header=None, sep=';', decimal=',')


roth_raw_data = pd.read_csv("./propositio/roth.csv", header=None, sep=';', decimal=',')



In [251]:
def clean_data(array_raw_data):
    global begin_date
    # concat is first
    conc = True
    df = array_raw_data.iloc[1:]
    df.columns = cols
    df_temp = df.drop(["Date", 'DXY'], axis=1)
    df_temp = df_temp.astype(float)
    df_temp["Date"] = pd.to_datetime(df["Date"])
   
    # df_temp["Date"] = pd.to_datetime(df["Date"])

    return df_temp

In [257]:

df_temp = clean_data(creil_raw_data)
df = prepare_data(df_temp)
df = df[['RR', 'FFM', 'PMERM', 'GLOT', 'UM', 'TN', 'TX','TM']]
df.head()

Dataframes saved...


,RR,FFM,PMERM,GLOT,UM,TN,TX,TM
12,0.6,2.6,1010.0,161.0,86.0,-1.5,0.1,-0.7
13,0.8,2.2,1001.9,120.0,93.0,-2.2,3.9,0.8
14,0.0,2.2,1005.3,368.0,93.0,0.3,5.3,1.7
15,0.0,1.7,1021.3,114.0,95.0,-1.8,2.9,1.4
16,9.0,4.8,1012.3,96.0,93.0,1.8,8.6,3.9


In [258]:
df.head()

,RR,FFM,PMERM,GLOT,UM,TN,TX,TM
12,0.6,2.6,1010.0,161.0,86.0,-1.5,0.1,-0.7
13,0.8,2.2,1001.9,120.0,93.0,-2.2,3.9,0.8
14,0.0,2.2,1005.3,368.0,93.0,0.3,5.3,1.7
15,0.0,1.7,1021.3,114.0,95.0,-1.8,2.9,1.4
16,9.0,4.8,1012.3,96.0,93.0,1.8,8.6,3.9


In [259]:
X_std = StandardScaler().fit_transform(df)
Y_pca = pca.fit_transform(X_std)


In [260]:

pca.explained_variance_ratio_

array([0.45509695, 0.22771439, 0.12818092, 0.08397957, 0.07634338,
       0.01812896, 0.00954186, 0.00101397])

In [261]:
df.head()

,RR,FFM,PMERM,GLOT,UM,TN,TX,TM
12,0.6,2.6,1010.0,161.0,86.0,-1.5,0.1,-0.7
13,0.8,2.2,1001.9,120.0,93.0,-2.2,3.9,0.8
14,0.0,2.2,1005.3,368.0,93.0,0.3,5.3,1.7
15,0.0,1.7,1021.3,114.0,95.0,-1.8,2.9,1.4
16,9.0,4.8,1012.3,96.0,93.0,1.8,8.6,3.9


In [239]:
pd.DataFrame(pca.components_, columns=['RR', 'TN', 'TX','TM', 'PMERM','FFM','UM','GLOT'], )

,RR,TM,PMERM,FFM,UM,GLOT
0,-0.215197,0.456371,0.207884,-0.140539,-0.560957,0.606445
1,0.530813,0.335635,-0.612632,0.442336,-0.151081,0.108545
2,0.467230,0.330294,-0.033733,-0.763093,0.297359,0.027016
3,0.608906,-0.045896,0.732172,0.293602,-0.069564,0.003321
4,0.286451,-0.736203,-0.210326,-0.286431,-0.416355,0.276260
5,0.028341,0.160868,0.004053,-0.184477,-0.629197,-0.737144
